## Dynamic Programming

In [121]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [122]:
from collections import defaultdict

In [123]:
price = defaultdict(int)

In [124]:
for i, p in enumerate(original_price): 
    price[i + 1] = p

In [125]:
price[30]

0

## Get the max splitting by enumerate

### 背景知识：python is a language that is function oriental 面向函数的语言

In [9]:
def example(f,arg):
    return f(arg)
def add_ten(num):
    return num+10
def mul_ten(num):
    return num*10
operations = [add_ten,mul_ten]
for f in operations:
    print(example(f,100))

110
1000


In [81]:
called_time = defaultdict(int)
def get_call_times(f):
    result = f()
    print ('function: {} called once!'.format(f.__name__))
    called_time[f.__name__]+=1
    return result
def some_function_1():
    print('I am function 1')
get_call_times(some_function_1)
called_time

I am function 1
function: some_function_1 called once!


defaultdict(int, {'some_function_1': 1})

#### 我们来观察一下不同的r(n)被调用了多少次

In [92]:
call_time_with_arg = defaultdict(int)
def r(n):
    fname = r.__name__  ##我们在解决切管子的问题时，实际并不关心这行代码
    call_time_with_arg[(fname,n)] +=1 ##我们在解决切管子的问题时，实际并不关心这行代码
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])
r(4)
call_time_with_arg
from collections import Counter
Counter(call_time_with_arg).most_common()

[(('r', 1), 18), (('r', 2), 6), (('r', 3), 2), (('r', 4), 1)]

In [93]:
del call_time_with_arg

#### 我们实际并不希望执行fname=r.__name__ 和call_time_with_arg[(fname,n)]+=1 这两行代码。===》介绍python中装饰器的用法来解决


In [142]:
## 此处定义get_call_time（f） 后期可以@get_call_time 直接调用这一函数，我们不希望执行的那两行代码不必出现在r(n)函数中
from collections import defaultdict
call_time_with_arg = defaultdict(int)
def get_call_time(f):

    def wrap(n):
        """
        a help line: I am a wrap
        """
        print ('I can count')
        result = f(n)
        call_time_with_arg[(f.__name__,n)]+=1
        return result
    return wrap

In [139]:
def add_ten(num):
    return num+10
add_ten = get_call_time(add_ten)
add_ten(80)  ##此时add_ten这一函数不止有加10的功能，还可以额外执行call_time_with_arg调用次数的追踪

I can count


90

In [140]:
@get_call_time  ##这一模块功能和上面的相同，通过@直接实现
def add_twenty(num): return num+20
add_twenty(9)

I can count


29

In [115]:
del call_time_with_arg

#### 对比加上装饰器和不加装饰器时help显示的信息

In [143]:
from collections import defaultdict
call_time_with_arg = defaultdict(int)
@get_call_time
def r(n):
    """
    Args: n is the iron length
    return the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])
help(r)
call_time_with_arg
from collections import Counter
Counter(call_time_with_arg).most_common()

Help on function wrap in module __main__:

wrap(n)
    a help line: I am a wrap



[]

#### @wraps 帮助解决上述问题

In [148]:
from functools import wraps

In [220]:
called_time_with_arg = defaultdict(int)
def get_call_time(f):
    """@param f is a function"""
    @wraps(f)   ##此处添加一行
    def wrap(n):
        """Haha I am warp"""
        # print ('I can count!')
        result = f(n)
        called_time_with_arg[(f.__name__,n)]+=1
        return result
    return wrap

from collections import defaultdict
@get_call_time
def r(n):
    """
    Args: n is the iron length
    return the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])
help(r)
r(4)
called_time_with_arg


Help on function r in module __main__:

r(n)
    Args: n is the iron length
    return the max revenue



defaultdict(int, {('r', 1): 18, ('r', 2): 6, ('r', 3): 2, ('r', 4): 1})

In [218]:
del called_time_with_arg

#### cache 定义already_calculated作为缓存 存储已经计算过的r（n）从而提升运行效率 （call time会全变成1次）

In [326]:
already_calculated = {}
def memo(f):
    @wraps(f)
    def _wrap(arg):
        result = None
        if arg in already_calculated:
            result = already_calculated[arg]
        else:
            result = f(arg)
            already_calculated[arg] = result
        return result
    return _wrap

In [310]:

call_time_with_arg = defaultdict(int)
def get_call_time(f):
    """@param f is a function"""
    @wraps(f)   ##此处添加一行
    def w(n):
        """Haha I am warp"""
        #print ('I can count!')
        result = f(n)
        call_time_with_arg[(f.__name__,n)]+=1
        return result
    return w


In [314]:
@memo
@get_call_time
def r(n):
    """
    Args: n is the iron length
    return the max revenue
    """
    return max([price[n]]+[r(i)+r(n-i) for i in range(1,n)])

r(45)  ## 可以看到使用memo后，call time就全变1了！！


141

In [315]:
call_time_with_arg

defaultdict(int,
            {('r', 1): 1,
             ('r', 2): 1,
             ('r', 3): 1,
             ('r', 4): 1,
             ('r', 5): 1,
             ('r', 6): 1,
             ('r', 7): 1,
             ('r', 8): 1,
             ('r', 9): 1,
             ('r', 10): 1,
             ('r', 11): 1,
             ('r', 12): 1,
             ('r', 13): 1,
             ('r', 14): 1,
             ('r', 15): 1,
             ('r', 16): 1,
             ('r', 17): 1,
             ('r', 18): 1,
             ('r', 19): 1,
             ('r', 20): 1,
             ('r', 21): 1,
             ('r', 22): 1,
             ('r', 23): 1,
             ('r', 24): 1,
             ('r', 25): 1,
             ('r', 26): 1,
             ('r', 27): 1,
             ('r', 28): 1,
             ('r', 29): 1,
             ('r', 30): 1,
             ('r', 31): 1,
             ('r', 32): 1,
             ('r', 33): 1,
             ('r', 34): 1,
             ('r', 35): 1,
             ('r', 36): 1,
             ('r', 3

In [268]:
del call_time_with_arg

### 切管子问题 目前已经可以求得最高收入，下一步：显示最高收入对应的切割策略

In [317]:
solution = {}

In [322]:
memo.already_calculated = {}

In [327]:
@memo
def r(n):
    max_price, max_split = max(
        [(price[n],0)]+[(r(i)+r(n-i),i) for i in range(i,n-1)],key = lambda x: x[0]
    )
    solution[n] = (n-max_split,max_split)
    return max_price

In [328]:
r(38)

118

In [329]:
solution

{10: (10, 0),
 8: (8, 0),
 11: (11, 0),
 7: (7, 0),
 2: (2, 0),
 12: (2, 10),
 6: (6, 0),
 3: (3, 0),
 13: (2, 11),
 5: (5, 0),
 4: (4, 0),
 14: (3, 11),
 15: (2, 13),
 16: (3, 13),
 18: (3, 15),
 17: (6, 11),
 9: (9, 0),
 19: (6, 13),
 20: (10, 10),
 21: (11, 10),
 22: (11, 11),
 23: (13, 10),
 24: (13, 11),
 25: (14, 11),
 26: (15, 11),
 28: (17, 11),
 27: (16, 11),
 29: (18, 11),
 30: (19, 11),
 31: (21, 10),
 32: (22, 10),
 33: (22, 11),
 34: (24, 10),
 35: (24, 11),
 36: (25, 11),
 38: (27, 11)}

#### 最后一步 显示完整的最优切分方式  --所查表的解析 parse solution

In [330]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [331]:
r(234)

743

In [333]:
parse_solution(86)

[11, 11, 11, 11, 11, 11, 10, 10]

## Dynamic Programming

+ 1. Overlapping Subproblems
+ 2. Overlapping computing saved in a table
+ 3. Parse solution